## Setup



1. make sure we have access to use GPU

In [ ]:
!nvidia-smi

2. install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

## Prepare dataset

1. create train-val split

In [ ]:
import os
import shutil
import random

%pip install tqdm --upgrade
from tqdm.notebook import tqdm

2. connect to google drive and mount it (for dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

3. define files path

In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

# dataset location in drive
dataset_folder = '/content/drive/MyDrive/yolov8/ai-trackware'
dataset_source = dataset_folder +'/data/'

4. split dataset into train and test set (80% to train and 20% to test)

In [ ]:
def train_test_split(path,neg_path=None, split = 0.3):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 70% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split(dataset_source) ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

## Custom Training

In [ ]:
from datetime import datetime

now = datetime.now()

yolo_detection_model='yolov8s.pt'
dataset_config_path = dataset_folder + '/dataset.yaml'
training_output_folder = dataset_folder + '/training_output'
training_result_folder = 'result_yolov8s' + '_' + str(now.strftime("%Y%m%d_%H%M%S"))
training_result_path = os.path.join(training_output_folder, training_result_folder)

print('yolo_detection_model: ', yolo_detection_model)
print('dataset_config_path: ', dataset_config_path)
print('training_output_folder: ', training_output_folder)
print('training_result_folder: ', training_result_folder)
print('training_result_path: ', training_result_path)

In [ ]:
!yolo task=detect mode=train model={yolo_detection_model} data={dataset_config_path} epochs=50 imgsz=640 batch=8 project={training_output_folder} name={training_result_folder} 

## Inferencing

In [ ]:
prediction_model_path = os.path.join(training_output_folder, training_result_folder) + '/weights/best.pt'
prediction_output_folder = dataset_folder + '/output/' + training_result_folder
test_image_folder = dataset_folder + '/test'

print('prediction_model_path: ', prediction_model_path)
print('prediction_output_folder: ', prediction_output_folder)
print('test_image_folder: ', test_image_folder)

In [ ]:
!yolo task=detect mode=predict model={prediction_model_path} conf=0.55 source={test_image_folder}

In [ ]:
!cp -r /content/runs/detect/predict {prediction_output_folder}

## Validating Test Images

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{prediction_output_folder}/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")